# KNN Notes

So there are two types of, algorithm KNN regressor and classifier. 

KNN- finds the neighbours 

Classifier sees a point based on the point that was closest the KNN algorithm classifies point on it charechterisic like in classifier data is labled and put into a classifier algo and if a new point is enetered it will try to guess based on neighbours which classes does the data belong to.

Regressor based on closest point, and its neighbours it tries to guess the value. 


For KNN also there is a KNN regressor which has a input syntax of following
* KNN Regressor Syntax * 

sklearn.neighbors.KNeighborsRegressor(n_neighbors=5, *, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)[source]
 

Main Notes of the following points in the Algo 

- algorithm is component that decides which neighbours to take or not for distance computation, has 3 options Brute which will compute distance between all the neighbours and KDtree and Balltree (Can read)
- metric is distance metric which says what kind of distance will we take between parameters (Euclidian or Minkowski or any other)
- weights this decides how weightage to each varaibale you pass to KNN will be Suppose you pass lat, long and elevation what weightage to each component (NEED TO MAKE CUSTOM WEIGHT deciding matrix and try)

## So What is happening with KNN 
You tried to run the algo for different distance metrics and found that

So what is basically happening with KNN is that it takes 2/3/4/5 points based on the distance metric you have provided for eg euclidian so it will calculate lat,log,elev dist, and which ever point has the closest distance metric it uses that point to take average and find an imputed value. 

Now if you put weights into that while calculating the average the point more closer will be given more weightage than the one far away, (In our case we are getting optimum by just 2 close points so dosnt make much sense to play with weights over here) this is happeneing in the scene. 


rather then that I would say try to optimise the distance metric. Which would have some dynamic weights, to get the best 2/3/4/5 neighbours. 




# In conclusion. 

So in summary, adding more variables would just make the distance calcualtion metric more better but making the distance calculation meetric more accurate would just lead to better selection among the 7-8 afvaialable points. While currently optimal number of points for best KNN MSE is 2. So adding more parameters to just select 2 points would not lead to a significant increase. 

So maybe later you could try KNN with different variables  and stratergy or weighted KNN. 

NEED to check iterative imputer still. 

# Now all we can do is try weighted KNN built on own classes later on. also try iterative imputer with other estimators 


## Weighted KNN example

Below is an example of a simple weighted KNN Algorithm that was built it adjust the weights over 100 runs to optimise the imputation (It makes a class variable called KNN and does KNN in it that class where you can see data fit stores the varaibles and predict computes the distance between all points and does KNN finally ) and these weights are used to check MSE which uses gradient decent to see which direction to move into to get best MSE 

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Example dataset with features latitude, longitude, elevation, and target value
X = np.array([
    [35.6895, 139.6917, 44],  # Tokyo
    [34.0522, -118.2437, 305], # Los Angeles
    [40.7128, -74.0060, 10],  # New York
    [51.5074, -0.1278, 35],   # London
])
y = np.array([100, 200, 300, 150])  # Example target values

# Initial weights for the features
weights = np.ones(X.shape[1])

# Define a custom distance function that uses the weights
def weighted_distance(x1, x2, weights):
    return np.sqrt(np.sum(weights * (x1 - x2) ** 2))

# Define the weighted KNN regressor
class WeightedKNNRegressor:
    def __init__(self, n_neighbors=3, weights=None):
        self.n_neighbors = n_neighbors
        self.weights = weights

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = []
        for x in X:
            distances = np.array([weighted_distance(x, x_train, self.weights) for x_train in self.X_train])
            nearest_indices = np.argsort(distances)[:self.n_neighbors]
            nearest_values = self.y_train[nearest_indices]
            predictions.append(np.mean(nearest_values))
        return np.array(predictions)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize the weighted KNN regressor
weighted_knn = WeightedKNNRegressor(n_neighbors=3, weights=weights)

# Simple gradient descent to optimize weights
learning_rate = 0.01
n_iterations = 100

for iteration in range(n_iterations):
    weighted_knn.fit(X_train, y_train)
    y_pred = weighted_knn.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Iteration {iteration}, MSE: {mse}")

    # Compute gradient of weights
    gradients = np.zeros_like(weights)
    for j in range(len(weights)):
        weights[j] += 0.01  # Small perturbation
        perturbed_knn = WeightedKNNRegressor(n_neighbors=3, weights=weights)
        perturbed_knn.fit(X_train, y_train)
        perturbed_pred = perturbed_knn.predict(X_test)
        perturbed_mse = mean_squared_error(y_test, perturbed_pred)
        gradients[j] = (perturbed_mse - mse) / 0.01
        weights[j] -= 0.01  # Revert perturbation

    # Update weights
    weights -= learning_rate * gradients
    print(f"Updated weights: {weights}")

# Final prediction
weighted_knn.fit(X_train, y_train)
final_predictions = weighted_knn.predict(X_test)
print(f"Final predictions: {final_predictions}")


## Check IDW as a weighting stratergey in weighted KNN 